In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import requests

# Replace 'YOUR_API_KEY' with your actual Alpha Vantage API key
API_KEY = '3C5LTDSZAXVJ1DF4'

# Alpha Vantage API endpoint for real-time stock data
API_URL = 'https://www.alphavantage.co/query'

def fetch_stock_data(symbol):
    params = {
        'function': 'TIME_SERIES_INTRADAY',
        'symbol': symbol,
        'interval': '1min',
        'apikey': API_KEY
    }

    response = requests.get(API_URL, params=params)

    if response.status_code == 200:
        data = response.json()
        # Commenting out the print statement for data
        # print(data)
        return data['Time Series (1min)']
    else:
        print('Error:', response.status_code)
        return None


def predict_stock_prices(data):
    real_time_data = pd.DataFrame(data).T
    real_time_data.index = pd.to_datetime(real_time_data.index)
    real_time_data = real_time_data['4. close'].astype(float)
    real_time_data = real_time_data.asfreq('1T')  # Set the index frequency to 1 minute

    # ARIMA prediction
    model_arima = ARIMA(real_time_data, order=(5, 1, 0))
    model_fit_arima = model_arima.fit()
    predictions_arima = model_fit_arima.predict(start=len(real_time_data), end=len(real_time_data) + 10)

    return real_time_data, predictions_arima



def main():
    symbols = input("Enter stock symbols (comma-separated): ").split(',')
    for symbol in symbols:
        symbol = symbol.strip().upper()
        print(f"\nFetching data for {symbol}...")
        data = fetch_stock_data(symbol)
        if data:
            previous_day_data = pd.DataFrame(data).T
            previous_day_data.index = pd.to_datetime(previous_day_data.index)
            previous_day_close = previous_day_data.iloc[0]['4. close']

            real_time_data, predictions_arima = predict_stock_prices(data)

            # Visualize the predictions
            plt.figure(figsize=(12, 6))
            plt.plot(real_time_data, label='Real-Time Stock Prices')
            plt.axhline(y=previous_day_close, color='gray', linestyle='--', label='Previous Day Close')
            plt.plot(predictions_arima, label='ARIMA Predictions', color='red')
            plt.xlabel('Date')
            plt.ylabel('Stock Price (Close)')
            plt.legend()
            plt.title(f'Stock Prices and ARIMA Predictions for {symbol}')
            plt.show()

            # Print the predicted values
            print(f"\nARIMA Predicted Stock Prices for {symbol}:")
            print(predictions_arima)

if __name__ == "__main__":
    main()
